# MDE and Number of samples calculations

## 1. Minimum Detectable Effect (MDE)


https://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/BS704_Power/BS704_Power_print.html

In [2]:
import numpy as np
from scipy.stats import norm


def get_mde(a, b):
    mean_std =  (
          a.std()**2 / a.shape[0]
        + b.std()**2 / b.shape[0]
    )**0.5
    mde = norm.ppf(0.975) / a.mean() * mean_std
    return mde

In [3]:
70000* 0.0015

105.0

In [4]:
norm.ppf(0.975)

1.959963984540054

In [24]:
norm.ppf(0.8)

0.8416212335729143

In [25]:
def get_mde_binominal(n_samples, p_base, alpha=0.05, beta=0.8, relative_mde=False):
    z_alpha = norm.ppf(1-alpha/2)
    z_beta = norm.ppf(beta)
    
    sigma = p_base * (1 - p_base)
    mde = (z_alpha + z_beta) * np.sqrt(2 * sigma / n_samples)
    
    if relative_mde:
        return mde /p_base
    else:
        return mde

In [26]:
num_samples = 70000
prob_conversion = 0.0015

# relative Minimum Detectable Effect
get_mde_binominal(num_samples, prob_conversion, relative_mde=True)

0.38636539366739764

In [29]:
# relative Minimum Detectable Effect
get_mde_binominal(num_samples, prob_conversion, relative_mde=False)

0.0005795480905010965

In [27]:
get_mde_binominal(num_samples, prob_conversion, beta=0.9, relative_mde=True)

0.44703599358331125

In [28]:
get_mde_binominal(num_samples, prob_conversion, beta=0.5, relative_mde=True)

0.27029777697456575

## 2. Number of samples

In [27]:
import numpy as np
from scipy.stats import norm


def get_num_samples_binomial(mde, p_base, alpha=0.05, beta=0.8, relative_mde=False):
    z_alpha = norm.ppf(1-alpha/2)
    z_beta = norm.ppf(beta)
    
    sigma = p_base * (1 - p_base)
    
    if relative_mde:
        mde = mde * p_base
        
    n_samples = 2 * sigma * ((z_alpha + z_beta)/ mde)**2
    
    return n_samples

In [16]:
0.05/36

0.001388888888888889

In [28]:
mde = 0.01
prob_conversion = 0.05
num_groups = 36
p_value = 0.05
alpha = p_value/ num_groups

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=False)

15494.55648640007

In [29]:
mde = 0.2
prob_conversion = 0.05
num_groups = 36
p_value = 0.05
alpha = p_value/ num_groups

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=True)

15494.556486400064

In [23]:
mde = 0.01
prob_conversion = 0.05
num_groups = 18
p_value = 0.05
alpha = p_value/ num_groups

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=False)

13956.838283519304

In [18]:
mde = 0.01
prob_conversion = 0.05
num_groups = 72
p_value = 0.05
alpha = p_value/ num_groups

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=False)

17025.466096419925

In [22]:
mde = 0.01
prob_conversion = 0.05
num_groups = 36
p_value = 0.05
alpha = 0.037

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=False)

8141.105452499457

In [17]:
mde = 0.01
prob_conversion = 0.07
#num_groups = 36
#p_value = 0.05
alpha = 0.05

get_num_samples_binomial(mde, prob_conversion, alpha, beta=0.8, relative_mde=False)

10219.241414122514

### Several experiment sequental false positive error

In [9]:
p_value = 0.05
num_groups = 36
num_weeks = 1

alpha = p_value / num_groups

def prob_error(alpha, num_weeks, num_groups):
    return 1 - (1 - alpha**num_weeks)**num_groups

In [10]:
p_value = 0.05
num_groups = 36
num_weeks = 1

alpha = p_value / num_groups

prob_error(alpha, num_weeks, num_groups)

0.04880363433838786

In [11]:
p_value = 0.01
num_groups = 36
num_weeks = 1

alpha = p_value / num_groups

prob_error(alpha, num_weeks, num_groups)

0.009951541573787304

In [12]:
p_value = 0.05
num_groups = 36
num_weeks = 2

alpha = p_value / num_groups

prob_error(alpha, num_weeks, num_groups)

6.944210020831676e-05

In [13]:
p_value = 0.01
num_groups = 36
num_weeks = 2

alpha = p_value / num_groups

prob_error(alpha, num_weeks, num_groups)

2.7777740249090854e-06

## Methods

In [1]:
def sample_size_binomial(mde, p, alpha=0.05, beta=0.1):
    """
        Определить размер групп для AB-теста с минимальным детектируемым эффектом p_1-p_0
        Значения в выборках: пропорции из [0, 1]
    """
    
    Z_alpha = st.norm.ppf(1 - alpha/2)
    Z_beta = st.norm.ppf(1 - beta)
    std = np.sqrt(p * (1 - p))
    N = 2 * (std * (Z_alpha + Z_beta) / mde) ** 2
    
    return int(N) + 1


def sample_size_continuous(mde, std, alpha=0.05, beta=0.1):    
    """
        Определить размер групп для AB-теста с минимальным детектируемым эффектом mde
        Значения в выборках: непрерывные

        Parameters
        ----------
        mde - минимальная разница средних, которую хотим детектировать
        std - предпосчитанная std для объединенной выборки
    """
    
    Z_alpha = st.norm.ppf(1 - alpha/2)
    Z_beta = st.norm.ppf(1 - beta)
    N = 2 * (std * (Z_alpha + Z_beta) / mde) ** 2
    
    return int(N) + 1

## 3. Linearization

In [2]:
def get_linearization_pvalue(
    g0, g1, metric_colname
):
    # g0 - объекты из контроля
    # g1 - объекты из таргета
    # metric_colname - метрика, сейчас сумма платежей объекта.

    k = (
        g0[metric_colname].sum()
        / g0['students_cnt'].sum()
    )
    g0['lin_rev'] = g0[metric_colname] - k * g0['students_cnt']
    g1['lin_rev'] = g1[metric_colname] - k * g1['students_cnt'] 
    pvalue = ttest_ind(
        g0.lin_rev,
        g1.lin_rev,
        equal_var = False
    )[1]
    
    return pvalue

def get_delta_method_pvalue(g0, g1, metric_colname):
    # g0 - объекты из контроля
    # g1 - объекты из таргета
    # metric_colname - метрика, сейчас сумма платежей объекта.
    
    # school = user
    # students_cnt = cnt of impressions
    # metric_colname = ctr on impression (for ex.)

    def ratio_metric_variance(
        numerator_by_user_array,
        denominator_by_user_array
    ): 
        X_d_mean = np.mean(denominator_by_user_array)
        X_n_mean = np.mean(numerator_by_user_array)
        X_d_variance = np.var(denominator_by_user_array)
        X_n_variance = np.var(numerator_by_user_array)
        X_d_n_covariance = np.cov(
            denominator_by_user_array,
            numerator_by_user_array,
            bias=True
        )[0][1]
        return (
            (X_n_variance)/(X_d_mean**2)
            - 2*X_n_mean*X_d_n_covariance/(X_d_mean**3)
            + (X_n_mean**2) * (X_d_variance)/(X_d_mean**4)
        )
    
    con_var = ratio_metric_variance(
        g0[metric_colname], g0['students_cnt']
    )
    exp_var = ratio_metric_variance(
        g1[metric_colname], g1['students_cnt']
    )

    con_mean = g0[metric_colname].sum() / g0['students_cnt'].sum()
    exp_mean = g1[metric_colname].sum() / g1['students_cnt'].sum()

    z_stats = (exp_mean - con_mean) / np.sqrt(
        con_var/g0.shape[0] + exp_var/g1.shape[0])
    pvalue = 2 * np.minimum(
        scipy.stats.norm(0, 1).cdf(z_stats),
        1 - scipy.stats.norm(0, 1).cdf(z_stats)
    )
    return pvalue

In [ ]:
import itertools
import numpy as np
from scipy import stats
from statsmodels.stats import proportion


def continuous_ttest(control_values, test_values):
    """T-test for the means of two independent samples

    control_values: List, list of values in control group (user-level values)
    test_values: List, list of values in test group (user-level values)

    return:
        delta - difference in means
        p_value.
    """
    _, p_value = stats.ttest_ind(control_values, test_values)
    delta = round((np.mean(test_values) - np.mean(control_values)) /
                  np.abs(np.mean(control_values)) * 100, 2
                  )
    return delta, p_value


def linearized_ttest(control_values, test_values):
    """T-test for the linearized metrics of two samples

    control_values: List[List], list of arrays of session-values in control group (session-level values)
    test_values: List[List], list of arrays of session-values in test group (session-level values)

    return:
        delta - difference in means
        p_value.
    """
    control_x = np.array([np.sum(row) for row in control_values])
    control_y = np.array([len(row) for row in control_values])
    test_x = np.array([np.sum(row) for row in test_values])
    test_y = np.array([len(row) for row in test_values])
    coef = np.sum(control_x) / np.sum(control_y)
    control_lin = control_x - coef * control_y
    test_lin = test_x - coef * test_y
    _, p_value = stats.ttest_ind(control_lin, test_lin)
    ### Delta for means:
    delta = round(
        (np.mean(list(itertools.chain(*test_values))) - np.mean(list(itertools.chain(*control_values)))) /
        np.abs(np.mean(list(itertools.chain(*control_values)))) * 100, 2
    )
    return delta, p_value


def proportion_ztest(control_values, test_values):
    """Z-test for proportions of two independent samples

    control_values: List[np.array], list of arrays of session-values in control group (session-level values)
    test_values: List[np.array], list of arrays of session-values in test group (session-level values)

    return:
        delta - difference in means
        p_value.
    """
    counts = np.array([sum(control_values), sum(test_values)])
    nobs = np.array([len(control_values), len(test_values)])
    _, p_value = proportion.proportions_ztest(counts, nobs)
    delta = round((np.mean(test_values) - np.mean(control_values)) /
                  np.abs(np.mean(control_values)) * 100, 2
                 )
    return delta, p_value